<a href="https://colab.research.google.com/github/rsaeed15/Drug-Target-Interaction-/blob/main/ML_Protein_Structure_Classification_GraphData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p><img alt="Colaboratory logo" height="45px" src="/img/colab_favicon.ico" align="left" hspace="10px" vspace="0px"></p>

<h1>What is Colaboratory?</h1>

Colaboratory, or "Colab" for short, allows you to write and execute Python in your browser, with 
- Zero configuration required
- Free access to GPUs
- Easy sharing

Whether you're a **student**, a **data scientist** or an **AI researcher**, Colab can make your work easier. Watch [Introduction to Colab](https://www.youtube.com/watch?v=inN8seMm7UI) to learn more, or just get started below!

## **Getting started**

The document you are reading is not a static web page, but an interactive environment called a **Colab notebook** that lets you write and execute code.

For example, here is a **code cell** with a short Python script that computes a value, stores it in a variable, and prints the result:

In [1]:
!pip install stellargraph
from stellargraph import datasets
from IPython.display import display, HTML
dataset = datasets.PROTEINS()
graphs, graph_labels = dataset.load() 

     |████████████████████████████████| 435 kB 4.8 MB/s 


In [2]:
# Convert from StellarGraph format to numpy adj matrices
adjs = [graph.to_adjacency_matrix().A for graph in graphs]
# Convert labels from Pandas.Series to numpy array
labels = graph_labels.to_numpy(dtype=int) 

In [3]:
import numpy as np
import networkx as nx
metrics = []
for adj in adjs:
 G = nx.from_numpy_matrix(adj)
 # basic properties
 num_edges = G.number_of_edges()
 # clustering measures
 cc = nx.average_clustering(G)
 # global efficiency
 eff_glb = nx.global_efficiency(G)
 # local efficiency
 eff_loc=nx.local_efficiency(G)
 # transitivity
 trans=nx.transitivity(G)
 # assortativity coefficient
 corr=nx.degree_pearson_correlation_coefficient(G)
 # shortest path length
 #short=nx.average_shortest_path_length(G)
 metrics.append([num_edges, cc, eff_glb, eff_loc,
                 trans, corr])

/usr/local/lib/python3.7/dist-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


In [4]:
arr = np.array(metrics)

In [5]:
type(arr)

numpy.ndarray

In [6]:
import pandas as pd
df = pd.DataFrame(arr)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1113 entries, 0 to 1112
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       1113 non-null   float64
 1   1       1113 non-null   float64
 2   2       1113 non-null   float64
 3   3       1113 non-null   float64
 4   4       1113 non-null   float64
 5   5       1100 non-null   float64
dtypes: float64(6)
memory usage: 52.3 KB


In [8]:
df.corr()

,0,1,2,3,4,5
0,1.000000,-0.461393,-0.582589,-0.451309,-0.467969,0.355310
1,-0.461393,1.000000,0.622076,0.988957,0.987188,-0.494751
2,-0.582589,0.622076,1.000000,0.574248,0.613753,-0.572907
3,-0.451309,0.988957,0.574248,1.000000,0.974874,-0.442880
4,-0.467969,0.987188,0.613753,0.974874,1.000000,-0.463544
5,0.355310,-0.494751,-0.572907,-0.442880,-0.463544,1.000000


In [9]:
df.dropna(axis=0,inplace=True)

In [10]:
df[5].isnull().sum()

0

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1100 entries, 0 to 1112
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       1100 non-null   float64
 1   1       1100 non-null   float64
 2   2       1100 non-null   float64
 3   3       1100 non-null   float64
 4   4       1100 non-null   float64
 5   5       1100 non-null   float64
dtypes: float64(6)
memory usage: 60.2 KB


In [12]:
#df[5].fillna(value=0, method=None,inplace=True)

In [13]:
metric=df.to_numpy()

In [14]:
type(labels)

numpy.ndarray

In [15]:
labels = np.delete(labels, np.s_[1100:1113], 0)

In [16]:
len(labels)

1100

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(metric, labels, test_size=0.3, random_state=42)

In [18]:
from sklearn import svm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
clf = svm.SVC()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print('Accuracy', accuracy_score(y_test,y_pred))
print('Precision', precision_score(y_test,y_pred))
print('Recall', recall_score(y_test,y_pred))
print('F1-score', f1_score(y_test,y_pred))

Accuracy 0.7242424242424242
Precision 0.7341772151898734
Recall 0.8613861386138614
F1-score 0.7927107061503417


In [19]:
#y_pred = clf.predict([[7, 0.8, 0.85]])
y_pred = clf.predict([X_test[1]]) 

In [20]:
y_pred

array([2])

In [21]:
df2 = pd.DataFrame(labels)

In [22]:
frames = [df2, df]

In [23]:
result = pd.concat(frames)

In [24]:
result.corr()

,0,1,2,3,4,5
0,1.000000,-0.456378,-0.588353,-0.445842,-0.464942,0.355310
1,-0.456378,1.000000,0.597987,0.989881,0.987925,-0.494751
2,-0.588353,0.597987,1.000000,0.557924,0.578788,-0.572907
3,-0.445842,0.989881,0.557924,1.000000,0.979386,-0.442880
4,-0.464942,0.987925,0.578788,0.979386,1.000000,-0.463544
5,0.355310,-0.494751,-0.572907,-0.442880,-0.463544,1.000000


To execute the code in the above cell, select it with a click and then either press the play button to the left of the code, or use the keyboard shortcut "Command/Ctrl+Enter". To edit the code, just click the cell and start editing.

Variables that you define in one cell can later be used in other cells:

Colab notebooks allow you to combine **executable code** and **rich text** in a single document, along with **images**, **HTML**, **LaTeX** and more. When you create your own Colab notebooks, they are stored in your Google Drive account. You can easily share your Colab notebooks with co-workers or friends, allowing them to comment on your notebooks or even edit them. To learn more, see [Overview of Colab](/notebooks/basic_features_overview.ipynb). To create a new Colab notebook you can use the File menu above, or use the following link: [create a new Colab notebook](http://colab.research.google.com#create=true).

Colab notebooks are Jupyter notebooks that are hosted by Colab. To learn more about the Jupyter project, see [jupyter.org](https://www.jupyter.org).

## Data science

With Colab you can harness the full power of popular Python libraries to analyze and visualize data. The code cell below uses **numpy** to generate some random data, and uses **matplotlib** to visualize it. To edit the code, just click the cell and start editing.

You can import your own data into Colab notebooks from your Google Drive account, including from spreadsheets, as well as from Github and many other sources. To learn more about importing data, and how Colab can be used for data science, see the links below under [Working with Data](#working-with-data).

## Machine learning

With Colab you can import an image dataset, train an image classifier on it, and evaluate the model, all in just [a few lines of code](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb). Colab notebooks execute code on Google's cloud servers, meaning you can leverage the power of Google hardware, including [GPUs and TPUs](#using-accelerated-hardware), regardless of the power of your machine. All you need is a browser.

Colab is used extensively in the machine learning community with applications including:
- Getting started with TensorFlow
- Developing and training neural networks
- Experimenting with TPUs
- Disseminating AI research
- Creating tutorials

To see sample Colab notebooks that demonstrate machine learning applications, see the [machine learning examples](#machine-learning-examples) below.

## More Resources

### Working with Notebooks in Colab
- [Overview of Colaboratory](/notebooks/basic_features_overview.ipynb)
- [Guide to Markdown](/notebooks/markdown_guide.ipynb)
- [Importing libraries and installing dependencies](/notebooks/snippets/importing_libraries.ipynb)
- [Saving and loading notebooks in GitHub](https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb)
- [Interactive forms](/notebooks/forms.ipynb)
- [Interactive widgets](/notebooks/widgets.ipynb)
- <img src="/img/new.png" height="20px" align="left" hspace="4px" alt="New"></img>
 [TensorFlow 2 in Colab](/notebooks/tensorflow_version.ipynb)

<a name="working-with-data"></a>
### Working with Data
- [Loading data: Drive, Sheets, and Google Cloud Storage](/notebooks/io.ipynb) 
- [Charts: visualizing data](/notebooks/charts.ipynb)
- [Getting started with BigQuery](/notebooks/bigquery.ipynb)

### Machine Learning Crash Course
These are a few of the notebooks from Google's online Machine Learning course. See the [full course website](https://developers.google.com/machine-learning/crash-course/) for more.
- [Intro to Pandas DataFrame](https://colab.research.google.com/github/google/eng-edu/blob/main/ml/cc/exercises/pandas_dataframe_ultraquick_tutorial.ipynb)
- [Linear regression with tf.keras using synthetic data](https://colab.research.google.com/github/google/eng-edu/blob/main/ml/cc/exercises/linear_regression_with_synthetic_data.ipynb)


<a name="using-accelerated-hardware"></a>
### Using Accelerated Hardware
- [TensorFlow with GPUs](/notebooks/gpu.ipynb)
- [TensorFlow with TPUs](/notebooks/tpu.ipynb)

<a name="machine-learning-examples"></a>

## Machine Learning Examples

To see end-to-end examples of the interactive machine learning analyses that Colaboratory makes possible, check out these  tutorials using models from [TensorFlow Hub](https://tfhub.dev).

A few featured examples:

- [Retraining an Image Classifier](https://tensorflow.org/hub/tutorials/tf2_image_retraining): Build a Keras model on top of a pre-trained image classifier to distinguish flowers.
- [Text Classification](https://tensorflow.org/hub/tutorials/tf2_text_classification): Classify IMDB movie reviews as either *positive* or *negative*.
- [Style Transfer](https://tensorflow.org/hub/tutorials/tf2_arbitrary_image_stylization): Use deep learning to transfer style between images.
- [Multilingual Universal Sentence Encoder Q&A](https://tensorflow.org/hub/tutorials/retrieval_with_tf_hub_universal_encoder_qa): Use a machine learning model to answer questions from the SQuAD dataset.
- [Video Interpolation](https://tensorflow.org/hub/tutorials/tweening_conv3d): Predict what happened in a video between the first and the last frame.
